In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
from tensorflow.keras import layers
from google.colab import drive

In [ ]:
drive.flush_and_unmount()
drive.mount('/content/gdrive')
#drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# check number of files in each directory

import glob

classnames = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
totalcount = 0
for name in classnames:
  data_files = glob.glob("/content/gdrive/My Drive/Capstone/Dataset/combined_set/" + name + "/*")
  count = 0
  for data_file in data_files:
    count += 1
    totalcount += 1
  print(name + ":" + str(count))
print("total number of images:" + str(totalcount))

In [ ]:
data_dir="/content/gdrive/My Drive/Capstone/Dataset/master_training_set"
data_dir = pathlib.Path(data_dir)

In [ ]:
batch_size = 32
img_height = 200
img_width = 200

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

In [ ]:
test_dir="/content/gdrive/My Drive/Capstone/Dataset/Jing"
test_dir = pathlib.Path(test_dir)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
test_loss, test_acc = model.evaluate(test_ds, verbose=1)

In [ ]:
#data_dir= "/content/gdrive/My Drive/Capstone/Dataset/master_training_set"
data_dir = "/content/gdrive/My Drive/Capstone/Dataset/combined_set"
data_dir = pathlib.Path(data_dir)

batch_size = 32
img_height = 200
img_width = 200

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  follow_links=True)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  follow_links=True)
class_names = train_ds.class_names

print(class_names)

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

# AUTOTUNE in keras is -1 as a constant
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

num_classes = 29
model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['sparse_categorical_accuracy'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)

model.save("gdrive/My Drive/Capstone/models/nov_6_2020")

# Note: something is wrong with saving the model!
# https://www.tensorflow.org/guide/saved_model

# test_dir="/content/gdrive/My Drive/Capstone/Dataset/Jing"
# test_dir = pathlib.Path(test_dir)
# test_ds = tf.keras.preprocessing.image_dataset_from_directory(
#   test_dir,
#   image_size=(img_height, img_width),
#   batch_size=batch_size)
# test_loss, test_acc = model.evaluate(test_ds, verbose=1)

Found 17707 files belonging to 29 classes.
Using 14166 files for training.
Found 17707 files belonging to 29 classes.
Using 3541 files for validation.
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
Epoch 1/5
443/443 [==============================] - 147s 331ms/step - loss: 2.1036 - sparse_categorical_accuracy: 0.4251 - val_loss: 0.8746 - val_sparse_categorical_accuracy: 0.7509
Epoch 2/5
443/443 [==============================] - 148s 334ms/step - loss: 0.4429 - sparse_categorical_accuracy: 0.8762 - val_loss: 0.4476 - val_sparse_categorical_accuracy: 0.8752
Epoch 3/5
443/443 [==============================] - 146s 331ms/step - loss: 0.1325 - sparse_categorical_accuracy: 0.9620 - val_loss: 0.5270 - val_sparse_categorical_accuracy: 0.8709
Epoch 4/5
443/443 [==============================] - 149s 335ms/step - loss: 0.0855 - sparse_categorical_accuracy: 0.9776 - val_loss: 0.2982 -

In [ ]:
# Model fit -> evaluate -> save -> load: consistently produces ~2-4% accuracy
eval_model = tf.keras.models.load_model("gdrive/My Drive/Capstone/models/nov_6_2020")

# Model fit -> evaluate: produces good accuracy on results
test_loss, test_acc = model.evaluate(val_ds, verbose=1)
test_loss, test_acc = eval_model.evaluate(val_ds, verbose=1)

In [ ]:
eval_model = tf.keras.models.load_model("gdrive/My Drive/Capstone/models/nov_6_2020")
dir = "/content/gdrive/My Drive/Capstone/Dataset/test_set"
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  dir,
  image_size=(img_height, img_width),
  batch_size=batch_size)
test_names = test_ds.class_names

test_loss, test_acc = eval_model.evaluate(test_ds, verbose=1)

In [4]:
# attempt to train loaded model on additional ASL Alphabet training data
model = tf.keras.models.load_model("gdrive/My Drive/Capstone/models/nov_6_2020")

asl_alphabet_dir = "/content/gdrive/My Drive/Capstone/Dataset/ASL Alphabet/asl_alphabet_train/asl_alphabet_train"
asl_alphabet_dir = pathlib.Path(asl_alphabet_dir)

batch_size = 32
img_height = 200
img_width = 200

asl_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  asl_alphabet_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  follow_links=True)

asl_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  asl_alphabet_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  follow_links=True)
class_names = asl_train_ds.class_names

model.fit(
  asl_train_ds,
  validation_data=asl_val_ds,
  epochs=5
)

Found 86000 files belonging to 29 classes.
Using 68800 files for training.
Found 86000 files belonging to 29 classes.
Using 17200 files for validation.
Epoch 1/5
2150/2150 [==============================] - 19046s 9s/step - loss: 0.8208 - sparse_categorical_accuracy: 0.7727 - val_loss: 0.1862 - val_sparse_categorical_accuracy: 0.9399
Epoch 2/5
2150/2150 [==============================] - 761s 354ms/step - loss: 0.1120 - sparse_categorical_accuracy: 0.9632 - val_loss: 0.0717 - val_sparse_categorical_accuracy: 0.9762
Epoch 3/5
2150/2150 [==============================] - 759s 353ms/step - loss: 0.0670 - sparse_categorical_accuracy: 0.9779 - val_loss: 0.0762 - val_sparse_categorical_accuracy: 0.9747
Epoch 4/5
2150/2150 [==============================] - 720s 335ms/step - loss: 0.0483 - sparse_categorical_accuracy: 0.9849 - val_loss: 0.1124 - val_sparse_categorical_accuracy: 0.9692
Epoch 5/5
2150/2150 [==============================] - 691s 321ms/step - loss: 0.0399 - sparse_categorical_ac

In [5]:
model.save("gdrive/My Drive/Capstone/models/nov_8_2020")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: gdrive/My Drive/Capstone/models/nov_8_2020/assets


In [6]:
eval_model = tf.keras.models.load_model("gdrive/My Drive/Capstone/models/nov_8_2020")
dir = "/content/gdrive/My Drive/Capstone/Dataset/test_set"
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  dir,
  image_size=(img_height, img_width),
  batch_size=batch_size)
test_names = test_ds.class_names

test_loss, test_acc = eval_model.evaluate(test_ds, verbose=1)

Found 8702 files belonging to 29 classes.
272/272 [==============================] - 1739s 6s/step - loss: 27.3610 - sparse_categorical_accuracy: 0.1230
